In [2]:
import pandas as pd
import pickle

df = pd.read_pickle(r"C:\Users\AJain7\Downloads\village_feature_resnet50.pkl")

inception_train = pd.read_csv(r"C:\Users\AJain7\OneDrive - Stryker\Personal\Projects\Satellite Project\4. Village_level\2. Cross-Sectional\Classification\3Bands\ensemble\Inception_imbalance\msw\Train_Results-04.csv",delim_whitespace=True, header=None)
inception_test = pd.read_csv(r"C:\Users\AJain7\OneDrive - Stryker\Personal\Projects\Satellite Project\4. Village_level\2. Cross-Sectional\Classification\3Bands\ensemble\Inception_imbalance\msw\Test_Results-04.csv",delim_whitespace=True, header=None)
resnet_train = pd.read_csv(r"C:\Users\AJain7\OneDrive - Stryker\Personal\Projects\Satellite Project\4. Village_level\2. Cross-Sectional\Classification\3Bands\ensemble\Resnet_imbalance\msw\Train_Results-04.csv",delim_whitespace=True, header=None)
resnet_test = pd.read_csv(r"C:\Users\AJain7\OneDrive - Stryker\Personal\Projects\Satellite Project\4. Village_level\2. Cross-Sectional\Classification\3Bands\ensemble\Resnet_imbalance\msw\Test_Results-04.csv",delim_whitespace=True, header=None)

In [3]:
cols = ['actual_y', 'predicted_y', 'village_code']

inception_train.columns = cols
inception_test.columns = cols
resnet_train.columns = cols
resnet_test.columns = cols

inception_train['actual_y'] = inception_train['actual_y'].astype(int)
inception_train['predicted_y'] = inception_train['predicted_y'].astype(int)
inception_train['village_code'] = inception_train['village_code'].astype(int)

inception_test['actual_y'] = inception_test['actual_y'].astype(int)
inception_test['predicted_y'] = inception_test['predicted_y'].astype(int)
inception_test['village_code'] = inception_test['village_code'].astype(int)

resnet_train['actual_y'] = resnet_train['actual_y'].astype(int)
resnet_train['predicted_y'] = resnet_train['predicted_y'].astype(int)
resnet_train['village_code'] = resnet_train['village_code'].astype(int)

resnet_test['actual_y'] = resnet_test['actual_y'].astype(int)
resnet_test['predicted_y'] = resnet_test['predicted_y'].astype(int)
resnet_test['village_code'] = resnet_test['village_code'].astype(int)

In [16]:
counter = 0
df_list = []
for key, value in df.items():
        temp_df = pd.DataFrame(value).transpose()
        temp_df['village_code'] = int(key)
        df_list.append(temp_df)
        counter = counter+1
        
        if(counter // 5000):
            print(counter)
            counter=0

5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000


In [19]:
final_df = pd.concat(df_list)

In [23]:
final_df.to_csv('ensemble_partial_features.csv', index=False)

In [66]:
# resnet = pd.concat([resnet_train, resnet_test], axis=0)

# inception = pd.concat([inception_train, inception_test], axis=0)

# resnet.columns = ['actual_y', 'predicted_y_resnet', 'village_code']

# inception.columns = ['actual_y', 'predicted_y_inception', 'village_code']

# resnet.to_csv('resnet_msw.csv', index=False)
# inception.to_csv('inception_msw.csv', index=False)

In [56]:
# resincep = resnet.merge(inception, how='left', on='village_code')

# resin = pd.merge(resnet, inception, how='left', on='village_code')

# resin.drop_duplicates(subset=['predicted_y_resnet', 'village_code', 'predicted_y_inception'],inplace=True)

In [73]:
resin = pd.read_csv('resnet_inception_msw.csv')

In [79]:
input_data = resin.merge(final_df, how='left', on='village_code')

In [80]:
input_data.shape

(165504, 2052)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import metrics
from sklearn.metrics import f1_score, accuracy_score

Using TensorFlow backend.


In [2]:
def F1(y_true, y_pred):
    def recall(y_true, y_pred):
        #Recall metric: Only computes a batch-wise average of recall.
        #Computes the recall, a metric for multi-label classification of how many relevant items are selected
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        #Precision metric: Only computes a batch-wise average of precision.
        #Computes the precision, a metric for multi-label classification of how many selected items are relevant.
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [3]:
final_df = pd.read_csv('ensemble_partial_features.csv')

resin = pd.read_csv('resnet_inception_msw.csv')

resin = resin[['village_code','actual_y', 'predicted_y_resnet', 'predicted_y_inception']]

input_data = resin.merge(final_df, how='left', on='village_code')

In [4]:
feature_df = input_data[input_data.columns[4:].tolist()] # normal features
target_col = 'actual_y'

inception = pd.get_dummies(input_data['predicted_y_inception'])
inception.columns = ['inception_0','inception_1','inception_2']
resnet = pd.get_dummies(input_data['predicted_y_resnet'])
resnet.columns = ['resnet_0', 'resnet_1','resnet_2']
categorical_df = pd.concat([inception, resnet], axis=1)

In [5]:
X = pd.concat([feature_df, categorical_df], axis=1)

In [6]:
# Target
y = input_data[target_col]
y_encoded = keras.utils.to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

In [8]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(1024,activation='relu', input_shape=(X.shape[1],)))
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              2104320   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
__________

In [10]:
optim = Adam(lr=0.001)

In [11]:
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy', F1])

In [12]:
model.fit(x=X_train, y=y_train,validation_split=0.1,batch_size=128,epochs=20, verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=4)])

Instructions for updating:
Use tf.cast instead.
Train on 134057 samples, validate on 14896 samples
Epoch 1/20
134057/134057 [==============================] - 138s 1ms/step - loss: 0.6668 - acc: 0.7216 - F1: 0.7022 - val_loss: 0.6167 - val_acc: 0.7504 - val_F1: 0.7441
Epoch 2/20
134057/134057 [==============================] - 137s 1ms/step - loss: 0.6229 - acc: 0.7443 - F1: 0.7376 - val_loss: 0.6523 - val_acc: 0.7438 - val_F1: 0.7440
Epoch 3/20
134057/134057 [==============================] - 133s 989us/step - loss: 0.6191 - acc: 0.7461 - F1: 0.7395 - val_loss: 0.6082 - val_acc: 0.7504 - val_F1: 0.7441
Epoch 4/20
134057/134057 [==============================] - 128s 955us/step - loss: 0.6179 - acc: 0.7456 - F1: 0.7401 - val_loss: 0.6104 - val_acc: 0.7504 - val_F1: 0.7441
Epoch 5/20
134057/134057 [==============================] - 134s 1ms/step - loss: 0.6147 - acc: 0.7464 - F1: 0.7410 - val_loss: 0.6106 - val_acc: 0.7504 - val_F1: 0.7441
Epoch 6/20
134057/134057 [=====================

In [13]:
model.evaluate(x=X_test, y=y_test)

16551/16551 [==============================] - 5s 311us/step


[0.6365862235037065, 0.7213461422288564, 0.7390158618838324]

In [14]:
predictions = model.predict_classes(X_test)

In [15]:
inverse_y_test = np.array([np.argmax(i) for i in y_test])

In [16]:
print(f1_score(inverse_y_test, predictions,  average='weighted'))
print(f1_score(inverse_y_test, predictions,  average='micro'))
print(f1_score(inverse_y_test, predictions,  average='macro'))

0.7327138686624244
0.7213461422270557
0.7039288958570759
